Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

Preparing the dataset

In [2]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))])

trainset = torchvision.datasets.MNIST(root = './data',train = True,download = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset,batch_size = 4,shuffle = True, num_workers = 2)

testset = torchvision.datasets.MNIST(root = './data',train = False,download = True,transform = transform)
testloader = torch.utils.data.DataLoader(testset,batch_size = 4, shuffle = False, num_workers = 2)

classes = tuple(str(i) for i in range(10))


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:02<00:00, 4.59MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 132kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:01<00:00, 1.25MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 10.4MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



CNN Architecture

In [3]:
class Network(nn.Module) :
  def __init__(self):
    super(Network,self).__init__()

    self.Conv1 = nn.Conv2d(1,32,kernel_size = 3,padding = 1)
    self.bn1 = nn.BatchNorm2d(32)
    self.Conv2 = nn.Conv2d(32,64,kernel_size = 3,padding = 1)
    self.bn2 = nn.BatchNorm2d(64)
    self.Conv3 = nn.Conv2d(64,128,kernel_size = 3, padding = 1)
    self.bn3 = nn.BatchNorm2d(128)

    self.pool = nn.MaxPool2d(2,2)
    self.gap = nn.AdaptiveAvgPool2d((1,1))

    self.fc = nn.Linear(128,10)
    self.dropout = nn.Dropout(0.5)

  def forward(self,x):
      x = self.pool(F.relu(self.bn1(self.Conv1(x))))
      x = self.pool(F.relu(self.bn2(self.Conv2(x))))
      x = self.pool(F.relu(self.bn3(self.Conv3(x))))
      x = self.gap(x)
      x = torch.flatten(x,1)
      x = self.dropout(x)
      x = self.fc(x)
      return x

net = Network()

Loss Function and Optimizer

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=0.0001)

Training the Network

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)
for epoch in range(2):
  net.train()
  running_loss = 0.0

  for i,data in enumerate(trainloader,0):
    inputs,labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    outputs = net(inputs)
    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    if i % 2000 == 1999:
      print(f"[Epoch {epoch + 1}, Batch {i + 1}] Loss: {running_loss / 2000:.3f}")
      running_loss = 0.0
print("Training Finished")

[Epoch 1, Batch 2000] Loss: 0.178
[Epoch 1, Batch 4000] Loss: 0.169
[Epoch 1, Batch 6000] Loss: 0.161
[Epoch 1, Batch 8000] Loss: 0.152
[Epoch 1, Batch 10000] Loss: 0.151
[Epoch 1, Batch 12000] Loss: 0.133
[Epoch 1, Batch 14000] Loss: 0.128
[Epoch 2, Batch 2000] Loss: 0.125
[Epoch 2, Batch 4000] Loss: 0.116
[Epoch 2, Batch 6000] Loss: 0.098
[Epoch 2, Batch 8000] Loss: 0.106
[Epoch 2, Batch 10000] Loss: 0.109
[Epoch 2, Batch 12000] Loss: 0.099
[Epoch 2, Batch 14000] Loss: 0.099
Training Finished


Testing the Network

In [7]:
correct = 0
total = 0

net.eval()
with torch.no_grad():
  for data in testloader:
    inputs,labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = net(inputs)
    _, predicted = torch.max(outputs.data,1)

    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f} %')

Accuracy of the network on the 10000 test images: 98.71 %


Optical Extension

In [10]:
from torchvision import models

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(28, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

net = models.resnet18(pretrained = True)
net.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 10)

net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=0.0001)

for epoch in range(2):
  net.train()
  running_loss = 0.0

  for i,data in enumerate(trainloader,0):
    inputs,labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    outputs = net(inputs)
    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    if i % 2000 == 1999:
      print(f"[Epoch {epoch + 1}, Batch {i + 1}] Loss: {running_loss / 2000:.3f}")
      running_loss = 0.0
print("Training Finished")

correct = 0
total = 0

net.eval()
with torch.no_grad():
  for data in testloader:
    inputs,labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = net(inputs)
    _, predicted = torch.max(outputs.data,1)

    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f} %')

[Epoch 1, Batch 2000] Loss: 1.292
[Epoch 1, Batch 4000] Loss: 0.761
[Epoch 1, Batch 6000] Loss: 0.533
[Epoch 1, Batch 8000] Loss: 0.422
[Epoch 1, Batch 10000] Loss: 0.363
[Epoch 1, Batch 12000] Loss: 0.311
[Epoch 1, Batch 14000] Loss: 0.270
[Epoch 2, Batch 2000] Loss: 0.226
[Epoch 2, Batch 4000] Loss: 0.212
[Epoch 2, Batch 6000] Loss: 0.174
[Epoch 2, Batch 8000] Loss: 0.168
[Epoch 2, Batch 10000] Loss: 0.158
[Epoch 2, Batch 12000] Loss: 0.148
[Epoch 2, Batch 14000] Loss: 0.158
Training Finished
Accuracy of the network on the 10000 test images: 91.51 %
